In [194]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
#from selenium.webdriver.support.expected_conditions import element_selection_state_to_be
#import matplotlib.pyplot as plt
import base64
#import matplotlib.pyplot as plt
import io
import os
from math import floor

## web crawling packages
import time
from datetime import date
from datetime import datetime
import datetime
import requests
from lxml import html
import csv
import urllib.request
from html_table_parser.parser import HTMLTableParser
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import selenium

## scrapping game id's from espn

In [2]:
# //*[@id="401326436"]
# //*[@id="401326436"]
# /html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[1]/div/section

In [3]:
# /html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[1]/div/section

In [163]:
# start chrome driver
# chromedriver_autoinstaller.install()

In [162]:
# webdriver_options = webdriver.ChromeOptions()

In [195]:
##temp 1 week at a time
# driver_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

# update the driver path to driver_path
# driver = webdriver.Chrome('c:/Users/rober/venv/ml/Lib/site-packages/chromedriver_autoinstaller/120/chromedriver.exe')
# driver = webdriver.Chrome(driver_path)

driver.get('https://www.espn.com/nfl/scoreboard/_/week/15/year/2023/seasontype/2')

## game id's
games = driver.find_elements(By.CLASS_NAME, 'Scoreboard')
game_list = []

for i in range(len(games)):
    # print(games[i].get_attribute("id"))
    game_list.append(games[i].get_attribute("id"))

# print(game_list)
game_id_df = pd.DataFrame(game_list, columns=['id'])
game_id_df['url'] = 'https://www.espn.com/nfl/scoreboard/_/week/15/year/2023/seasontype/2'
game_id_df.head()

,id,url
0,401547593,https://www.espn.com/nfl/scoreboard/_/week/15/...
1,401547604,https://www.espn.com/nfl/scoreboard/_/week/15/...
2,401547607,https://www.espn.com/nfl/scoreboard/_/week/15/...
3,401547606,https://www.espn.com/nfl/scoreboard/_/week/15/...
4,401547605,https://www.espn.com/nfl/scoreboard/_/week/15/...


In [196]:
game_id_df.to_csv('predictions/previous_week_games.csv', index=False)

In [197]:
game_stats_all = pd.read_csv('data/game_stats_all.csv')
print(game_stats_all.shape)
game_stats_all['id'] = game_stats_all['id'].astype(str)

# show rows where id is in game_list
game_stats_all_df = game_stats_all[~game_stats_all['id'].isin(game_list)]
print(game_stats_all_df.shape)

game_id_all = pd.read_csv('data/game_id_all.csv')
print(game_id_all.shape)
game_id_all['id'] = game_id_all['id'].astype(str)

# show rows where id is in game_list
game_id_all_df = game_id_all[~game_id_all['id'].isin(game_list)]
print(game_id_all_df.shape)

game_scores = pd.read_csv('data/game_scores.csv')
print(game_scores.shape)
game_scores['id'] = game_scores['id'].astype(str)

# show rows where id is in game_list
game_scores_df = game_scores[~game_scores['id'].isin(game_list)]
print(game_scores_df.shape)

(5843, 51)
(5827, 51)
(6077, 2)
(6061, 2)
(5940, 5)
(5924, 5)


In [165]:
# game_stats_all_df.to_csv('data/game_stats_all.csv', index=False)
# game_id_all_df.to_csv('data/game_id_all.csv', index=False)
# game_scores_df.to_csv('data/game_scores.csv', index=False)

## starting code for the loop here

In [94]:
url_list = []

year_list = ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
# year_list = ['2021', '2022']

for i in year_list:
    year = str(i)

    for i in range(1,19):
        week = str(i)
        url_list.append([year, week])

url_df = pd.DataFrame(url_list, columns = ['year', 'week'])
url_df['url'] = "https://www.espn.com/nfl/scoreboard/_/week/" + url_df['week'] + "/year/" + url_df['year'] + "/seasontype/2"
print(url_df.shape)

# post season
post_season_list = []
for i in year_list:
    year = str(i)

    for i in range(1,4):
        week = str(i)
        post_season_list.append([year, week])
ps_url_df = pd.DataFrame(post_season_list, columns = ['year', 'week'])
ps_url_df['url'] = "https://www.espn.com/nfl/scoreboard/_/week/" + ps_url_df['week'] + "/year/" + ps_url_df['year'] + "/seasontype/3"

# concat url_df and ps_url_df
url_df = pd.concat([url_df, ps_url_df], axis = 0)
print(url_df.shape)

url_df.head()

(396, 3)
(462, 3)


,year,week,url
0,2002,1,https://www.espn.com/nfl/scoreboard/_/week/1/y...
1,2002,2,https://www.espn.com/nfl/scoreboard/_/week/2/y...
2,2002,3,https://www.espn.com/nfl/scoreboard/_/week/3/y...
3,2002,4,https://www.espn.com/nfl/scoreboard/_/week/4/y...
4,2002,5,https://www.espn.com/nfl/scoreboard/_/week/5/y...


In [35]:
# url_df = url_df[11:33]
# url_df

In [67]:
# url_df.head(20)

In [95]:
# games[i].get_attribute("id")

In [157]:
# url_df = url_df.head(10) ## for testing
game_id_df = pd.DataFrame(columns=['id', 'url'])

# driver_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

for i in url_df['url']:
    driver.get(i)
    url = i

    game_id_temp = []

    games = driver.find_elements(By.CLASS_NAME, 'Scoreboard')

    for j in range(len(games)):
        # print(games[i].get_attribute("id"))
        
        game_id_temp.append(games[j].get_attribute("id"))
        
    
    game_id_df_temp = pd.DataFrame(game_id_temp, columns=['id'])
    game_id_df_temp['url'] = url

    # concat to game_id_df
    game_id_df = pd.concat([game_id_df, game_id_df_temp]).reset_index(drop=True)

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=120.0.6099.109)
Stacktrace:
	GetHandleVerifier [0x00DF6E73+174291]
	(No symbol) [0x00D20AC1]
	(No symbol) [0x00A36E8A]
	(No symbol) [0x00A2A152]
	(No symbol) [0x00A28983]
	(No symbol) [0x00A28DD8]
	(No symbol) [0x00A38EB7]
	(No symbol) [0x00A9A9D2]
	(No symbol) [0x00A8700C]
	(No symbol) [0x00A9A104]
	(No symbol) [0x00A86DA6]
	(No symbol) [0x00A61034]
	(No symbol) [0x00A61F8D]
	GetHandleVerifier [0x00E949CC+820268]
	sqlite3_dbdata_init [0x00F54EBE+652494]
	sqlite3_dbdata_init [0x00F548D9+650985]
	sqlite3_dbdata_init [0x00F4962C+605244]
	sqlite3_dbdata_init [0x00F5586B+654971]
	(No symbol) [0x00D2FEBC]
	(No symbol) [0x00D28428]
	(No symbol) [0x00D2854D]
	(No symbol) [0x00D15858]
	BaseThreadInitThunk [0x76437BA9+25]
	RtlInitializeExceptionChain [0x773FBD2B+107]
	RtlClearBits [0x773FBCAF+191]


In [166]:
print(game_id_df.shape)
game_id_df.tail()

(16, 2)


,id,url
11,401547597,https://www.espn.com/nfl/scoreboard/_/week/15/...
12,401547596,https://www.espn.com/nfl/scoreboard/_/week/15/...
13,401547600,https://www.espn.com/nfl/scoreboard/_/week/15/...
14,401547602,https://www.espn.com/nfl/scoreboard/_/week/15/...
15,401547601,https://www.espn.com/nfl/scoreboard/_/week/15/...


In [167]:
game_id_df.to_csv('data/game_id_catch_up.csv', index=False)

#### adhoc step - compare to games we have data for to reduce scraping effort

In [169]:
# import game_id_all.csv
game_id_df_all = pd.read_csv('data/game_id_all.csv')
print(game_id_df_all.shape)
game_id_df_all.head()

(6061, 2)


,id,url
0,220905019,https://www.espn.com/nfl/scoreboard/_/week/1/y...
1,220908002,https://www.espn.com/nfl/scoreboard/_/week/1/y...
2,220908003,https://www.espn.com/nfl/scoreboard/_/week/1/y...
3,220908004,https://www.espn.com/nfl/scoreboard/_/week/1/y...
4,220908005,https://www.espn.com/nfl/scoreboard/_/week/1/y...


In [170]:
# import game_id_all.csv
game_stats_all = pd.read_csv('data/game_stats_all.csv')
print(game_stats_all.shape)
game_stats_all.head()

(5827, 53)


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,fourth_down_away,total_plays_away,total_yards_away,total_drives_away,yards_per_play_away,...,red_zone_att_home,penalty_home,turnovers_home,Fumbles lost.1,int_thrown_home.1,defensive_td_home,possession_home,id,Unnamed: 1,Unnamed: 27
0,13,7,5,1,4-12,0-0,51,279,13,5.5,...,0-6,10-80,3,0,3,0,32:28,220905019,NaN,NaN
1,18,11,4,3,2-8,0-0,47,266,17,5.7,...,0-8,10-82,3,1,2,0,39:08,220908002,NaN,NaN
2,19,9,9,1,5-13,0-0,62,368,17,5.9,...,0-6,4-33,2,1,1,0,28:29,220908003,NaN,NaN
3,27,9,16,2,6-10,0-0,65,401,11,6.2,...,0-5,9-57,1,0,1,0,22:12,220908004,NaN,NaN
4,24,12,10,2,5-11,0-0,59,470,20,8.0,...,0-10,4-38,1,1,0,0,29:44,220908005,NaN,NaN


In [171]:
game_stats_all['id'] = game_stats_all['id'].astype(str)

In [111]:
# mask game_id_df where id is not in game_stats_all
curr_games = game_stats_all['id'].unique()
all_games = game_id_df['id'].unique()

game_id_df_reduce = game_id_df[~game_id_df['id'].isin(curr_games)]
print(game_id_df_reduce.shape)
game_id_df_reduce.head()

(566, 2)


,id,url
349,231019015,https://www.espn.com/nfl/scoreboard/_/week/7/y...
659,241121005,https://www.espn.com/nfl/scoreboard/_/week/11/...
661,241121018,https://www.espn.com/nfl/scoreboard/_/week/11/...
789,250918027,https://www.espn.com/nfl/scoreboard/_/week/2/y...
833,251009014,https://www.espn.com/nfl/scoreboard/_/week/5/y...


## Scrape Game Odds

In [316]:
## hitting this api will show the current weeks games and the betting data behind each game

#driver.get('https://www.espn.com/nfl/scoreboard')

url = "https://site.web.api.espn.com/apis/v2/scoreboard/header?sport=football&league=nfl&region=us&lang=en&contentorigin=espn&buyWindow=1m&showAirings=buy%2Clive%2Creplay&showZipLookup=true&tz=America/New_York"

payload={}
headers = {
  'Accept': 'application/json',
  'Cookie': 'SWID=3750444F-BA06-4929-C99C-62333D7EE8A0'
}

response = requests.request("GET", url, headers=headers, data=payload)

response_text = json.loads(response.text)

game_df = json_normalize(response_text['sports'], ['leagues', 'events'])

## include this data after predictions to find valuable bets that show some different towhat vegas thinks

In [317]:
game_df.head()

,gamecastAvailable,playByPlayAvailable,commentaryAvailable,recent,id,competitionId,uid,date,timeValid,name,...,odds.homeTeamOdds.moneyLine,odds.homeTeamOdds.spreadOdds,odds.homeTeamOdds.team.id,odds.homeTeamOdds.team.abbreviation,odds.links,odds.moneylineWinner,odds.spreadWinner,odds.home.moneyLine,odds.away.moneyLine,fullStatus.type.altDetail
0,False,False,False,False,401326497,401326497,s:20~l:28~e:401326497~c:401326497,2021-11-28T18:00:00Z,True,Pittsburgh Steelers at Cincinnati Bengals,...,-180,-110.0,4,CIN,[],False,False,-180,155,NaN
1,False,False,False,False,401326499,401326499,s:20~l:28~e:401326499~c:401326499,2021-11-28T18:00:00Z,True,Tampa Bay Buccaneers at Indianapolis Colts,...,130,100.0,11,IND,[],False,False,130,-150,NaN
2,False,False,False,False,401326501,401326501,s:20~l:28~e:401326501~c:401326501,2021-11-28T18:00:00Z,True,Carolina Panthers at Miami Dolphins,...,100,-115.0,15,MIA,[],False,False,100,-120,NaN
3,False,False,False,False,401326502,401326502,s:20~l:28~e:401326502~c:401326502,2021-11-28T18:00:00Z,True,Tennessee Titans at New England Patriots,...,-340,-105.0,17,NE,[],False,False,-340,270,NaN
4,False,False,False,False,401326503,401326503,s:20~l:28~e:401326503~c:401326503,2021-11-28T18:00:00Z,True,Philadelphia Eagles at New York Giants,...,175,-110.0,19,NYG,[],False,False,175,-210,NaN


## scraping individual game data -- we just need to find a way we get scrap all the game id's and then pass them to the functions below

In [172]:
# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its
    # binary contents (HTTP Response Body)

    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [92]:
# defining the html contents of a URL.
xhtml = url_get_contents('https://www.espn.com/nfl/matchup?gameId=221110009').decode('utf-8')

# Defining the HTMLTableParser object
p = HTMLTableParser()

# feeding the html contents in the
# HTMLTableParser object
p.feed(xhtml)

# Now finally obtaining the data of
# the table required
#pprint(p.tables[1])

df_testing = pd.DataFrame(p.tables[1])
column_list = list(df_testing[0].values)

away = pd.DataFrame(df_testing[1]).T
away.columns = column_list
away = away.reset_index()
# away = away.drop(columns = ['index', 'Matchup'])
away = away.drop(columns = ['index'])
#away.drop(away.columns[[13]], axis=1, inplace=True)
away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            'Passing 1st downs': 'first_downs_passing_away',
                            'Rushing 1st downs': 'first_downs_rushing_away', 
                            '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            'Total Drives': 'total_drives_away', 
                            'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',  
                            'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away', 
                            'Rushing 1st downs': 'first_down_rushing_away', 
                            'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

home = pd.DataFrame(df_testing[2]).T
home.columns = column_list
home = home.reset_index()
# home = home.drop(columns = ['index', 'Matchup'])
home = home.drop(columns = ['index'])
#home.drop(home.columns[[13]], axis=1, inplace=True)
home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            'Passing 1st downs': 'first_downs_passing_home',
                            'Rushing 1st downs': 'first_downs_rushing_home', 
                            '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            'Total Drives': 'total_drives_home', 
                            'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',  
                            'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home', 
                            'Rushing 1st downs': 'first_down_rushing_home', 
                            'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

game_stat_temp = pd.concat([away, home], axis=1)
game_stat_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0                               1 non-null      object
 1   first_downs_away            1 non-null      object
 2   first_downs_passing_away    1 non-null      object
 3   first_down_rushing_away     1 non-null      object
 4   first_down_by_penalty_away  1 non-null      object
 5   third_down_away             1 non-null      object
 6   fourth_down_away            1 non-null      object
 7   total_plays_away            1 non-null      object
 8   total_yards_away            1 non-null      object
 9   total_drives_away           1 non-null      object
 10  yards_per_play_away         1 non-null      object
 11  passing_away                1 non-null      object
 12  comp_att_away               1 non-null      object
 13  yards_per_pass_away         1 non-null      object
 14

In [93]:
p.tables

[[['', '1', '2', '3', '4', 'T'],
  ['DET', '7', '0', '0', '7', '14'],
  ['GB', '3', '27', '10', '0', '40']],
 [['', '', ''],
  ['1st Downs', '13', '24'],
  ['Passing 1st downs', '10', '17'],
  ['Rushing 1st downs', '3', '7'],
  ['1st downs from penalties', '0', '0'],
  ['3rd down efficiency', '1-13', '8-17'],
  ['4th down efficiency', '1-4', '0-0'],
  ['Total Plays', '61', '78'],
  ['Total Yards', '362', '503'],
  ['Total Drives', '24', '18'],
  ['Yards per Play', '5.9', '6.4'],
  ['Passing', '236', '353'],
  ['Comp-Att', '20-44', '29-42'],
  ['Yards per pass', '5.4', '8.0'],
  ['Interceptions thrown', '2', '0'],
  ['Sacks-Yards Lost', '0-0', '2-10'],
  ['Rushing', '126', '150'],
  ['Rushing Attempts', '17', '34'],
  ['Yards per rush', '7.4', '4.4'],
  ['Red Zone (Made-Att)', '0-10', '0-7'],
  ['Penalties', '1-10', '8-53'],
  ['Turnovers', '2', '2'],
  ['Fumbles lost', '0', '2'],
  ['Interceptions thrown', '2', '0'],
  ['Defensive / Special Teams TDs', '0', '2'],
  ['Possession', '20:3

In [91]:
print('Level Two gets you each row...', p.tables[1][1])
print('row header...', p.tables[1][1][0])
print('away data point...', p.tables[1][1][1])
print('home data point...', p.tables[1][1][2])

Level Two gets you each row... ['1st Downs', '25', '19']
row header... 1st Downs
away data point... 25
home data point... 19


### need to just write a loop to go through this table and extract all points

In [173]:
game_stats_big_df = pd.DataFrame()
# game_id_df = game_id_df_reduce.copy(deep=True) ## only for catching up missing games

for i in game_id_df['id']:
    
    try:
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        # defining the html contents of a URL.
        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        # Now finally obtaining the data of
        # the table required
        df_testing = pd.DataFrame(p.tables[1])
        column_list = list(df_testing[0].values)

        away = pd.DataFrame(df_testing[1]).T
        away.columns = column_list
        away = away.reset_index(drop=True)
        # away = away.drop(columns = ['index', 'Matchup'])
        #away.drop(away.columns[[13]], axis=1, inplace=True)
        away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            'Passing 1st downs': 'first_downs_passing_away',
                            'Rushing 1st downs': 'first_downs_rushing_away', 
                            '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            'Total Drives': 'total_drives_away', 
                            'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',
                            'Fumbles Lost': 'fumbles_lost_away',  
                            'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away', 
                            'Rushing 1st downs': 'first_down_rushing_away', 
                            'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

        home = pd.DataFrame(df_testing[2]).T
        home.columns = column_list
        home = home.reset_index(drop=True)
        # home = home.drop(columns = ['index', 'Matchup'])
        #home.drop(home.columns[[13]], axis=1, inplace=True)
        home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            'Passing 1st downs': 'first_downs_passing_home',
                            'Rushing 1st downs': 'first_downs_rushing_home', 
                            '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            'Total Drives': 'total_drives_home', 
                            'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',
                            'Fumbles Lost': 'fumbles_lost_home',  
                            'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home', 
                            'Rushing 1st downs': 'first_down_rushing_home', 
                            'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

        game_stat_temp = pd.concat([away, home], axis=1)
        game_stat_temp['id'] = i
        game_stats_big_df = pd.concat([game_stats_big_df, game_stat_temp], axis=0).reset_index(drop=True)

    except Exception as e:
        print("Error:", e, "Game ID:", i)
    

In [175]:
# game_stats_big_df.head()

In [ ]:
# game_stats_big_df = game_stats_big_df.drop(columns = ['Team Stats'])
# game_stats_big_df.shape

(345, 51)

In [ ]:
# game_stats_big_df.columns

In [177]:
game_stats_big_df.to_csv('data/game_stats_catch_up.csv', index=False)

In [178]:
game_score_big_df = pd.DataFrame()

for i in game_id_df['id']:
    
    try:
        # defining the html contents of a URL.
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        df_score = pd.DataFrame(p.tables[0])
        df_score = pd.DataFrame(df_score.iloc[1: , :])
        team_column = df_score[0].to_frame()
        #team_column = team_column.rename(columns = {0: 'team'})
        #team_df = pd.DataFrame(team_column, columns = ['team'])
        score_column = df_score.iloc[: , -1].to_frame()
        #score_column = score_column.rename({: 'score'})
        #score_df = pd.DataFrame(score_column, columns = ['score'])
        df_score_new = pd.concat([team_column, score_column], axis=1)
        df_score_new.columns = ['team', 'score']

        away_score = pd.DataFrame(df_score_new.iloc[0]).T
        away_score = away_score.reset_index(drop=True)
        # away_score = away_score.drop(columns = ['index'])
        #away_score = away_score[['team', 'score']]
        away_score = away_score.rename(columns = {'team': 'away_team', 'score': 'away_score'})

        home_score = pd.DataFrame(df_score_new.iloc[1]).T
        home_score = home_score.reset_index(drop=True)
        # home_score = home_score.drop(columns = ['index'])
        #home_score = home_score[['team', 'score']]
        home_score = home_score.rename(columns = {'team': 'home_team', 'score': 'home_score'})

        game_score_temp = pd.concat([away_score, home_score], axis=1)
        game_score_temp['id'] = i
        #game_score_temp = game_score_temp.reset_index()
        #game_score_temp = game_score_temp.drop(columns = ['index'])
        # game_score_big_df = game_score_big_df.append(game_score_temp)
        game_score_big_df = pd.concat([game_score_big_df, game_score_temp], axis=0).reset_index(drop=True)

    except Exception as e:
        print('Error:', e, 'game_id=', i)

In [179]:
print(game_score_big_df.shape)
game_score_big_df.head()

(16, 5)


,away_team,away_score,home_team,home_score,id
0,LAC,21,LV,63,401547593
1,MIN,24,CIN,27,401547604
2,PIT,13,IND,30,401547607
3,DEN,17,DET,42,401547606
4,CHI,17,CLE,20,401547605


In [87]:
# i = '401437835' ## insert for new game id

# try:
#     # defining the html contents of a URL.
#     web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

#     xhtml = url_get_contents(web).decode('utf-8')

#     # Defining the HTMLTableParser object
#     p = HTMLTableParser()

#     # feeding the html contents in the
#     # HTMLTableParser object
#     p.feed(xhtml)

#     df_score = pd.DataFrame(p.tables[0])
#     df_score = pd.DataFrame(df_score.iloc[1: , :])
#     team_column = df_score[0].to_frame()
#     #team_column = team_column.rename(columns = {0: 'team'})
#     #team_df = pd.DataFrame(team_column, columns = ['team'])
#     score_column = df_score.iloc[: , -1].to_frame()
#     #score_column = score_column.rename({: 'score'})
#     #score_df = pd.DataFrame(score_column, columns = ['score'])
#     df_score_new = pd.concat([team_column, score_column], axis=1)
#     df_score_new.columns = ['team', 'score']

#     away_score = pd.DataFrame(df_score_new.iloc[0]).T
#     away_score = away_score.reset_index()
#     away_score = away_score.drop(columns = ['index'])
#     #away_score = away_score[['team', 'score']]
#     away_score = away_score.rename(columns = {'team': 'away_team', 'score': 'away_score'})

#     home_score = pd.DataFrame(df_score_new.iloc[1]).T
#     home_score = home_score.reset_index()
#     home_score = home_score.drop(columns = ['index'])
#     #home_score = home_score[['team', 'score']]
#     home_score = home_score.rename(columns = {'team': 'home_team', 'score': 'home_score'})

#     game_score_temp = pd.concat([away_score, home_score], axis=1)
#     game_score_temp['id'] = i
#     #game_score_temp = game_score_temp.reset_index()
#     #game_score_temp = game_score_temp.drop(columns = ['index'])
#     game_score_big_df = game_score_big_df.append(game_score_temp)

# except Exception as e:
#     print('Error:', e, 'game_id=', i)

In [88]:
# game_score_big_df

In [180]:
game_score_big_df.to_csv('data/game_scores_catch_up.csv', index=False)

## appendix - just needed to get everything together from seperate files

In [181]:
game_id_og = pd.read_csv('data/game_id_all.csv')
print(game_id_og.shape)
game_id_new = pd.read_csv('data/game_id_catch_up.csv')
# game_id_all = game_id_og.append(game_id_new)
game_id_all = pd.concat([game_id_og, game_id_new], axis=0)
print(game_id_all.shape)

game_scores_og = pd.read_csv('data/game_scores.csv')
print(game_scores_og.shape)
game_scores_new = pd.read_csv('data/game_scores_catch_up.csv')
# game_scores_all = game_scores_og.append(game_scores_new)
game_scores_all = pd.concat([game_scores_og, game_scores_new], axis=0)
print(game_scores_all.shape)

game_stats_og = pd.read_csv('data/game_stats_all.csv')
print(game_stats_og.shape)
game_stats_new = pd.read_csv('data/game_stats_catch_up.csv')
# game_stats_all = game_stats_og.append(game_stats_new)
game_stats_all = pd.concat([game_stats_og, game_stats_new], axis=0)
print(game_stats_all.shape)

(6061, 2)
(6077, 2)
(5924, 5)
(5940, 5)
(5827, 53)
(5843, 55)


In [187]:
game_id_all.to_csv('data/game_id_all.csv', index=False)
game_scores_all.to_csv('data/game_scores.csv', index=False)
game_stats_all.to_csv('data/game_stats_all.csv', index=False)

In [186]:
# drop any columns that start with Unnamed in game_stats_all
game_stats_all = game_stats_all.loc[:, ~game_stats_all.columns.str.startswith('Unnamed')]
game_stats_all.columns

Index(['first_downs_away', 'first_downs_passing_away',
       'first_down_rushing_away', 'first_down_by_penalty_away',
       'third_down_away', 'fourth_down_away', 'total_plays_away',
       'total_yards_away', 'total_drives_away', 'yards_per_play_away',
       'passing_away', 'comp_att_away', 'yards_per_pass_away',
       'int_thrown_away', 'sack_yards_lost_away', 'rushing_away',
       'rushing_att_away', 'yards_per_rush_away', 'red_zone_att_away',
       'penalty_away', 'turnovers_away', 'Fumbles lost', 'int_thrown_away.1',
       'defensive_td_away', 'possession_away', 'first_downs_home',
       'first_downs_passing_home', 'first_down_rushing_home',
       'first_down_by_penalty_home', 'third_down_home', 'fourth_down_home',
       'total_plays_home', 'total_yards_home', 'total_drives_home',
       'yards_per_play_home', 'passing_home', 'comp_att_home',
       'yards_per_pass_home', 'int_thrown_home', 'sack_yards_lost_home',
       'rushing_home', 'rushing_att_home', 'yards_per_rus

In [123]:
game_id_all = game_id_all.drop(columns = 'Unnamed: 0')
game_id_all.tail()

,id,url
561,401547756,https://www.espn.com/nfl/scoreboard/_/week/2/y...
562,401547757,https://www.espn.com/nfl/scoreboard/_/week/2/y...
563,401547758,https://www.espn.com/nfl/scoreboard/_/week/2/y...
564,401547379,https://www.espn.com/nfl/scoreboard/_/week/3/y...
565,401547380,https://www.espn.com/nfl/scoreboard/_/week/3/y...


In [127]:
# game_scores_all = game_scores_all.drop(columns = 'Unnamed: 0')
# game_stats_all = game_stats_all.drop(columns = 'Unnamed: 0')

In [72]:
game_id_all.to_csv('data/game_id_all.csv')

In [129]:
# game_stats_all = game_stats_all.drop(columns = 'Unnamed: 0')
# game_stats_all.head()

In [74]:
game_scores_all = game_scores_all.drop(columns = 'Unnamed: 0')
game_scores_all.head()

,away_team,away_score,home_team,home_score,id
0,SF,16,NYG,13,220905019
1,NYJ,37,BUF,31,220908002
2,MIN,23,CHI,27,220908003
3,SD,34,CIN,6,220908004
4,KC,40,CLE,39,220908005


In [188]:
game_details_combined = game_stats_all.merge(game_scores_all, on='id', how='inner')
print(game_details_combined.shape)
game_details_combined

(5927, 55)


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,fourth_down_away,total_plays_away,total_yards_away,total_drives_away,yards_per_play_away,...,turnovers_home,Fumbles lost.1,int_thrown_home.1,defensive_td_home,possession_home,id,away_team,away_score,home_team,home_score
0,13,7,5,1,4-12,0-0,51,279,13,5.5,...,3,0,3,0,32:28,220905019,SF,16,NYG,13
1,18,11,4,3,2-8,0-0,47,266,17,5.7,...,3,1,2,0,39:08,220908002,NYJ,37,BUF,31
2,19,9,9,1,5-13,0-0,62,368,17,5.9,...,2,1,1,0,28:29,220908003,MIN,23,CHI,27
3,27,9,16,2,6-10,0-0,65,401,11,6.2,...,1,0,1,0,22:12,220908004,SD,34,CIN,6
4,24,12,10,2,5-11,0-0,59,470,20,8.0,...,1,1,0,0,29:44,220908005,KC,40,CLE,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5922,14,9,3,2,3-14,3-5,58,297,12,5.1,...,2,2,0,0,35:53,401547597,WSH,20,LAR,28
5923,24,12,9,3,4-7,0-0,54,406,10,7.5,...,2,0,2,0,34:11,401547596,SF,45,ARI,29
5924,14,6,7,1,5-13,1-2,57,195,8,3.4,...,0,0,0,0,35:05,401547600,DAL,10,BUF,31
5925,25,9,14,2,5-12,1-1,69,396,10,5.7,...,2,2,0,0,25:41,401547602,BAL,23,JAX,7


In [131]:
game_details_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5922 entries, 0 to 5921
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   first_downs_away            5922 non-null   int64  
 1   first_downs_passing_away    5922 non-null   int64  
 2   first_down_rushing_away     5922 non-null   int64  
 3   first_down_by_penalty_away  5922 non-null   int64  
 4   third_down_away             5922 non-null   object 
 5   fourth_down_away            5922 non-null   object 
 6   total_plays_away            5922 non-null   int64  
 7   total_yards_away            5922 non-null   int64  
 8   total_drives_away           5922 non-null   int64  
 9   yards_per_play_away         5922 non-null   float64
 10  passing_away                5922 non-null   int64  
 11  comp_att_away               5922 non-null   object 
 12  yards_per_pass_away         5922 non-null   float64
 13  int_thrown_away             5922 

In [189]:
game_details_combined = game_details_combined.rename(columns={'Fumbles lost': 'fumbles_lost_away', 'Fumbles lost.1': 'fumbles_lost_home'})
game_details_combined = game_details_combined.drop(columns=['int_thrown_home.1', 'int_thrown_away.1'])
game_details_combined.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,fourth_down_away,total_plays_away,total_yards_away,total_drives_away,yards_per_play_away,...,penalty_home,turnovers_home,fumbles_lost_home,defensive_td_home,possession_home,id,away_team,away_score,home_team,home_score
0,13,7,5,1,4-12,0-0,51,279,13,5.5,...,10-80,3,0,0,32:28,220905019,SF,16,NYG,13
1,18,11,4,3,2-8,0-0,47,266,17,5.7,...,10-82,3,1,0,39:08,220908002,NYJ,37,BUF,31
2,19,9,9,1,5-13,0-0,62,368,17,5.9,...,4-33,2,1,0,28:29,220908003,MIN,23,CHI,27
3,27,9,16,2,6-10,0-0,65,401,11,6.2,...,9-57,1,0,0,22:12,220908004,SD,34,CIN,6
4,24,12,10,2,5-11,0-0,59,470,20,8.0,...,4-38,1,1,0,29:44,220908005,KC,40,CLE,39


In [135]:
game_details_combined.columns

Index(['first_downs_away', 'first_downs_passing_away',
       'first_down_rushing_away', 'first_down_by_penalty_away',
       'third_down_away', 'fourth_down_away', 'total_plays_away',
       'total_yards_away', 'total_drives_away', 'yards_per_play_away',
       'passing_away', 'comp_att_away', 'yards_per_pass_away',
       'int_thrown_away', 'sack_yards_lost_away', 'rushing_away',
       'rushing_att_away', 'yards_per_rush_away', 'red_zone_att_away',
       'penalty_away', 'turnovers_away', 'fumbles_lost_away',
       'defensive_td_away', 'possession_away', 'first_downs_home',
       'first_downs_passing_home', 'first_down_rushing_home',
       'first_down_by_penalty_home', 'third_down_home', 'fourth_down_home',
       'total_plays_home', 'total_yards_home', 'total_drives_home',
       'yards_per_play_home', 'passing_home', 'comp_att_home',
       'yards_per_pass_home', 'int_thrown_home', 'sack_yards_lost_home',
       'rushing_home', 'rushing_att_home', 'yards_per_rush_home',
       

In [192]:
# game_details_combined.columns

In [134]:
game_details_combined = game_details_combined.drop(columns=['Unnamed: 1', 'Unnamed: 27'])

In [193]:
game_details_combined.to_csv('data/nfl_historical_game_detail.csv', index=False)